# NBA Optimisation de Coorélation par Regression Linéaire

## Problématique : Nombre de Points gagnés pour chaque joueur dés 1950

### 1- Regression linéaire Nombres de points gagnées par rapport au nombre des matchs

Content

The data-set contains aggregate individual statistics for 67 NBA seasons. 
from basic box-score attributes such as points, assists, rebounds etc., to more advanced money-ball like features such as Value Over Replacement.

In [269]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [270]:
#charger les données 
#player_data.csv
player_data=pd.read_csv("player_data.csv")
#Players.csv
players=pd.read_csv("Players.csv")
#Seasons_Stats.csv
seasons_stats=pd.read_csv("Seasons_Stats.csv")
#vérifier si tous les individus ont bien un prix 


In [271]:
player_data.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [272]:
players.head()

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
0,0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [273]:
seasons_stats.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [274]:
players.drop(['Unnamed: 0'],axis='columns',inplace=True)
seasons_stats.drop(['Unnamed: 0'],axis='columns',inplace=True)


In [275]:
players.head()

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [276]:
seasons_stats.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [277]:
# Les POS ont des NAN

In [278]:
seasons_stats.dtypes 

Year      float64
Player     object
Pos        object
Age       float64
Tm         object
G         float64
GS        float64
MP        float64
PER       float64
TS%       float64
3PAr      float64
FTr       float64
ORB%      float64
DRB%      float64
TRB%      float64
AST%      float64
STL%      float64
BLK%      float64
TOV%      float64
USG%      float64
blanl     float64
OWS       float64
DWS       float64
WS        float64
WS/48     float64
blank2    float64
OBPM      float64
DBPM      float64
BPM       float64
VORP      float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
dtype: object

In [279]:
players.dtypes

Player          object
height         float64
weight         float64
collage         object
born           float64
birth_city      object
birth_state     object
dtype: object

In [280]:
player_data.dtypes

name           object
year_start      int64
year_end        int64
position       object
height         object
weight        float64
birth_date     object
college        object
dtype: object

In [281]:
players

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky
...,...,...,...,...,...,...,...
3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany


In [284]:
# Y = PTS
# X = 'G', 'height'

In [285]:
duplicas=seasons_stats.duplicated(keep=False,subset='Player')
seasons_stats[duplicas]

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24683,2017.0,Joe Young,PG,24.0,IND,33.0,0.0,135.0,11.4,0.433,...,0.733,1.0,16.0,17.0,15.0,4.0,0.0,5.0,5.0,68.0
24684,2017.0,Nick Young,SG,31.0,LAL,60.0,60.0,1556.0,14.1,0.588,...,0.856,25.0,112.0,137.0,58.0,37.0,14.0,36.0,137.0,791.0
24685,2017.0,Thaddeus Young,PF,28.0,IND,74.0,74.0,2237.0,14.9,0.562,...,0.523,131.0,318.0,449.0,122.0,114.0,30.0,96.0,135.0,814.0
24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,0.604,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0


In [286]:
doublons=seasons_stats[duplicas]
doublons

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24683,2017.0,Joe Young,PG,24.0,IND,33.0,0.0,135.0,11.4,0.433,...,0.733,1.0,16.0,17.0,15.0,4.0,0.0,5.0,5.0,68.0
24684,2017.0,Nick Young,SG,31.0,LAL,60.0,60.0,1556.0,14.1,0.588,...,0.856,25.0,112.0,137.0,58.0,37.0,14.0,36.0,137.0,791.0
24685,2017.0,Thaddeus Young,PF,28.0,IND,74.0,74.0,2237.0,14.9,0.562,...,0.523,131.0,318.0,449.0,122.0,114.0,30.0,96.0,135.0,814.0
24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,0.604,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0


In [287]:
seasons_stats2=doublons.groupby(['Player']).agg({'G':[sum],'PTS':[sum]})
seasons_stats2

,G,PTS
,sum,sum
Player,,
A.C. Green,1361.0,12928.0
A.J. English,151.0,1502.0
A.J. Guyton,80.0,442.0
A.J. Price,287.0,1656.0
Aaron Brooks,797.0,7839.0
...,...,...
Zelmo Beaty*,570.0,9107.0
Zendon Hamilton,146.0,611.0


In [288]:
seasons_stats2=seasons_stats2.reset_index()
seasons_stats2

,Player,G,PTS
,,sum,sum
0,A.C. Green,1361.0,12928.0
1,A.J. English,151.0,1502.0
2,A.J. Guyton,80.0,442.0
3,A.J. Price,287.0,1656.0
4,Aaron Brooks,797.0,7839.0
...,...,...,...
2972,Zelmo Beaty*,570.0,9107.0
2973,Zendon Hamilton,146.0,611.0
2974,Zoran Dragic,32.0,56.0


In [289]:
print(seasons_stats2)

                  Player       G      PTS
                             sum      sum
0             A.C. Green  1361.0  12928.0
1           A.J. English   151.0   1502.0
2            A.J. Guyton    80.0    442.0
3             A.J. Price   287.0   1656.0
4           Aaron Brooks   797.0   7839.0
...                  ...     ...      ...
2972        Zelmo Beaty*   570.0   9107.0
2973     Zendon Hamilton   146.0    611.0
2974        Zoran Dragic    32.0     56.0
2975      Zoran Planinic   148.0    561.0
2976  Zydrunas Ilgauskas   843.0  10976.0

[2977 rows x 3 columns]


In [290]:
seasons_stats2.columns=seasons_stats2.columns.droplevel(-1)
seasons_stats2

,Player,G,PTS
0,A.C. Green,1361.0,12928.0
1,A.J. English,151.0,1502.0
2,A.J. Guyton,80.0,442.0
3,A.J. Price,287.0,1656.0
4,Aaron Brooks,797.0,7839.0
...,...,...,...
2972,Zelmo Beaty*,570.0,9107.0
2973,Zendon Hamilton,146.0,611.0
2974,Zoran Dragic,32.0,56.0
2975,Zoran Planinic,148.0,561.0


In [291]:
seasons_stats2.head()

,Player,G,PTS
0,A.C. Green,1361.0,12928.0
1,A.J. English,151.0,1502.0
2,A.J. Guyton,80.0,442.0
3,A.J. Price,287.0,1656.0
4,Aaron Brooks,797.0,7839.0


In [292]:
inner_merged_total=pd.merge(seasons_stats2,seasons_stats,on=["Player"])
inner_merged_total

,Player,G_x,PTS_x,Year,Pos,Age,Tm,G_y,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_y
0,A.C. Green,1361.0,12928.0,1986.0,PF,22.0,LAL,82.0,1.0,1542.0,...,0.611,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,A.C. Green,1361.0,12928.0,1987.0,PF,23.0,LAL,79.0,72.0,2240.0,...,0.780,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,A.C. Green,1361.0,12928.0,1988.0,PF,24.0,LAL,82.0,64.0,2636.0,...,0.773,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,A.C. Green,1361.0,12928.0,1989.0,PF,25.0,LAL,82.0,82.0,2510.0,...,0.786,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,A.C. Green,1361.0,12928.0,1990.0,PF,26.0,LAL,82.0,82.0,2709.0,...,0.751,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23675,Zydrunas Ilgauskas,843.0,10976.0,2007.0,C,31.0,CLE,78.0,78.0,2130.0,...,0.807,242.0,357.0,599.0,123.0,48.0,98.0,141.0,257.0,925.0
23676,Zydrunas Ilgauskas,843.0,10976.0,2008.0,C,32.0,CLE,73.0,73.0,2222.0,...,0.802,263.0,419.0,682.0,104.0,34.0,120.0,135.0,247.0,1029.0
23677,Zydrunas Ilgauskas,843.0,10976.0,2009.0,C,33.0,CLE,65.0,65.0,1765.0,...,0.799,157.0,333.0,490.0,64.0,28.0,84.0,90.0,183.0,838.0
23678,Zydrunas Ilgauskas,843.0,10976.0,2010.0,C,34.0,CLE,64.0,6.0,1339.0,...,0.743,114.0,231.0,345.0,48.0,14.0,50.0,63.0,183.0,474.0


In [293]:
inner_merged_total.fillna(0,inplace=True)
inner_merged_total.isna().sum()

Player    0
G_x       0
PTS_x     0
Year      0
Pos       0
Age       0
Tm        0
G_y       0
GS        0
MP        0
PER       0
TS%       0
3PAr      0
FTr       0
ORB%      0
DRB%      0
TRB%      0
AST%      0
STL%      0
BLK%      0
TOV%      0
USG%      0
blanl     0
OWS       0
DWS       0
WS        0
WS/48     0
blank2    0
OBPM      0
DBPM      0
BPM       0
VORP      0
FG        0
FGA       0
FG%       0
3P        0
3PA       0
3P%       0
2P        0
2PA       0
2P%       0
eFG%      0
FT        0
FTA       0
FT%       0
ORB       0
DRB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS_y     0
dtype: int64

In [294]:
inner_merged_total

,Player,G_x,PTS_x,Year,Pos,Age,Tm,G_y,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_y
0,A.C. Green,1361.0,12928.0,1986.0,PF,22.0,LAL,82.0,1.0,1542.0,...,0.611,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,A.C. Green,1361.0,12928.0,1987.0,PF,23.0,LAL,79.0,72.0,2240.0,...,0.780,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,A.C. Green,1361.0,12928.0,1988.0,PF,24.0,LAL,82.0,64.0,2636.0,...,0.773,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,A.C. Green,1361.0,12928.0,1989.0,PF,25.0,LAL,82.0,82.0,2510.0,...,0.786,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,A.C. Green,1361.0,12928.0,1990.0,PF,26.0,LAL,82.0,82.0,2709.0,...,0.751,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23675,Zydrunas Ilgauskas,843.0,10976.0,2007.0,C,31.0,CLE,78.0,78.0,2130.0,...,0.807,242.0,357.0,599.0,123.0,48.0,98.0,141.0,257.0,925.0
23676,Zydrunas Ilgauskas,843.0,10976.0,2008.0,C,32.0,CLE,73.0,73.0,2222.0,...,0.802,263.0,419.0,682.0,104.0,34.0,120.0,135.0,247.0,1029.0
23677,Zydrunas Ilgauskas,843.0,10976.0,2009.0,C,33.0,CLE,65.0,65.0,1765.0,...,0.799,157.0,333.0,490.0,64.0,28.0,84.0,90.0,183.0,838.0
23678,Zydrunas Ilgauskas,843.0,10976.0,2010.0,C,34.0,CLE,64.0,6.0,1339.0,...,0.743,114.0,231.0,345.0,48.0,14.0,50.0,63.0,183.0,474.0


Alors , là j'ai les plus Important les Matchs Jouées : ( G_x )
Les Points gagnées par les Joueurs : ( PTS_x )

In [295]:
data_merge=inner_merged_total
X=data_merge[['G_x','MP','Age','ORB','DRB','PF']]
y=data_merge['PTS_x']

In [296]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [297]:
X_train.shape, y_train.shape

((16576, 6), (16576,))

In [298]:
X_test.shape, y_test.shape

((7104, 6), (7104,))

In [299]:
regr= LinearRegression()
model=regr.fit(X_train,y_train)

In [300]:
model.coef_ , model.intercept_

(array([ 12.54419405,   2.60853917, -10.31610411,  -8.12131864,
          2.68243524, -15.37104713]),
 -2000.0535344474902)

In [301]:
regr.predict(X_test)

array([ 7670.0139118 , -1856.29504399, 11790.89554765, ...,
        9583.72157787,  6652.66458584, 10482.01601901])

In [302]:
regr.score(X_test,y_test)

0.7483469964248707

In [303]:
Y_pred = regr.predict(X_train)
len(Y_pred)

16576

In [304]:
data_merge.head(10)

,Player,G_x,PTS_x,Year,Pos,Age,Tm,G_y,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_y
0,A.C. Green,1361.0,12928.0,1986.0,PF,22.0,LAL,82.0,1.0,1542.0,...,0.611,160.0,221.0,381.0,54.0,49.0,49.0,99.0,229.0,521.0
1,A.C. Green,1361.0,12928.0,1987.0,PF,23.0,LAL,79.0,72.0,2240.0,...,0.780,210.0,405.0,615.0,84.0,70.0,80.0,102.0,171.0,852.0
2,A.C. Green,1361.0,12928.0,1988.0,PF,24.0,LAL,82.0,64.0,2636.0,...,0.773,245.0,465.0,710.0,93.0,87.0,45.0,120.0,204.0,937.0
3,A.C. Green,1361.0,12928.0,1989.0,PF,25.0,LAL,82.0,82.0,2510.0,...,0.786,258.0,481.0,739.0,103.0,94.0,55.0,119.0,172.0,1088.0
4,A.C. Green,1361.0,12928.0,1990.0,PF,26.0,LAL,82.0,82.0,2709.0,...,0.751,262.0,450.0,712.0,90.0,66.0,50.0,116.0,207.0,1061.0
5,A.C. Green,1361.0,12928.0,1991.0,PF,27.0,LAL,82.0,21.0,2164.0,...,0.738,201.0,315.0,516.0,71.0,59.0,23.0,99.0,117.0,750.0
6,A.C. Green,1361.0,12928.0,1992.0,PF,28.0,LAL,82.0,53.0,2902.0,...,0.744,306.0,456.0,762.0,117.0,91.0,36.0,111.0,141.0,1116.0
7,A.C. Green,1361.0,12928.0,1993.0,PF,29.0,LAL,82.0,55.0,2819.0,...,0.739,287.0,424.0,711.0,116.0,88.0,39.0,116.0,149.0,1051.0
8,A.C. Green,1361.0,12928.0,1994.0,PF,30.0,PHO,82.0,55.0,2825.0,...,0.735,275.0,478.0,753.0,137.0,70.0,38.0,100.0,142.0,1204.0
9,A.C. Green,1361.0,12928.0,1995.0,SF,31.0,PHO,82.0,52.0,2687.0,...,0.732,194.0,475.0,669.0,127.0,55.0,31.0,114.0,146.0,916.0


In [305]:
data_merge.shape

(23680, 54)

Classement Des Joueurs

In [326]:
player_points=seasons_stats2
player_points

,Player,G,PTS
0,A.C. Green,1361.0,12928.0
1,A.J. English,151.0,1502.0
2,A.J. Guyton,80.0,442.0
3,A.J. Price,287.0,1656.0
4,Aaron Brooks,797.0,7839.0
...,...,...,...
2972,Zelmo Beaty*,570.0,9107.0
2973,Zendon Hamilton,146.0,611.0
2974,Zoran Dragic,32.0,56.0
2975,Zoran Planinic,148.0,561.0


In [329]:
points=player_points

In [334]:
points = player_points.sort_values("PTS",ascending=False)
points

,Player,G,PTS
1635,Kareem Abdul-Jabbar*,1560.0,38387.0
1637,Karl Malone*,1476.0,36928.0
2946,Wilt Chamberlain*,1118.0,33953.0
1739,Kobe Bryant,1346.0,33643.0
2029,Michael Jordan*,1072.0,32292.0
...,...,...,...
2247,Pavel Podkolzin,6.0,4.0
1034,Garth Joseph,8.0,4.0
2446,Ron Crevier,6.0,0.0
652,Darius Johnson-Odom,7.0,0.0


In [345]:
# create a list of our conditions
conditions = [
    (points['PTS'] <= 10000),
    (points['PTS'] > 10000) & (points['PTS'] <= 20000),
    (points['PTS'] > 20000) & (points['PTS'] <= 30000),
    (points['PTS'] > 30000)
    ]

# create a list of the values we want to assign for each condition
values = ['Mediocre', 'fair', 'Elite', 'Stars']

# create a new column and use np.select to assign values to it using our lists as arguments
points['tier'] = np.select(conditions, values)

# display updated DataFrame
points.head(10)

,Player,G,PTS,tier
1635,Kareem Abdul-Jabbar*,1560.0,38387.0,Stars
1637,Karl Malone*,1476.0,36928.0,Stars
2946,Wilt Chamberlain*,1118.0,33953.0,Stars
1739,Kobe Bryant,1346.0,33643.0,Stars
2029,Michael Jordan*,1072.0,32292.0,Stars
918,Eddie Johnson,2078.0,31852.0,Stars
811,Dirk Nowitzki,1394.0,30260.0,Stars
2577,Shaquille O'Neal*,1268.0,29428.0,Elite
1809,LeBron James,1061.0,28787.0,Elite
817,Dominique Wilkins*,1148.0,28591.0,Elite


In [348]:
points.tier.value_counts()

Mediocre    2590
fair         337
Elite         43
Stars          7
Name: tier, dtype: int64